# 사용자 데이터 예시
- 온도, 습도, 조도의 값을 받아 설문점수를 생성한다.
- 실내 적정값
    - 온도 : 18 ~ 20
    - 습도 : 40 ~ 60
    - 조도 : 700 ~ 1500 (사무실 기준)
- 사용자 생성자 매개변수
    - 온도, 습도, 조도 : 사용자가 가장 중요하게 생각하는 (본인의 점수 측정에 가장 영향을 많이 주는) 특징
    - 사용자 ID
- 사용자 데이터 기능
    - score : 현재 건물의 온도, 습도, 조도 값을 받아 본인이 중요하게 생각하는 특징이 적정 값에 가까울 수록 가장 높은 점수 부여 (20), 아니면 적정값에서 멀수록 가중치로 점수가 낮아지게, 나머지는 랜덤하게 높은 점수
    - save_score : 본인의 설문점수 기록 및 현재날짜도 같이 기록
    - 온도, 습도, 조도 모든 특징을 안 받은 사용자는 랜덤하게 점수를 부여한다. (성실도가 낮은 참여자)

In [132]:
import numpy as np
import pandas as pd
import random as ran
import datetime as dt

In [191]:
class User:
    def __init__(self, user_id,improved_features):
        self.user_id = user_id
        self.improved_features = improved_features
        self.survey = pd.DataFrame(columns=['temp','hum', 'lux','datetime', 'score'])
    
    def save_survey(self, temp, hum, lux ,score, date):
        self.survey = self.survey.append({
            "temp": temp,
            "hum": hum,
            "lux": lux,
            "datetime": date,
            "score": score
        }, ignore_index=True)
    
    def score(self, temp, hum, lux):
        _score = np.array([0,0,0])
        
        if len(self.improved_features) == 0:
            # 한줄인가?
            one_line = ran.randrange(0, 2)

            if one_line == 0:
                s = ran.randrange(5, 21, 5)
                for idx in range(0, len(_score)):
                    _score[idx] = s
            else:
                for idx in range(0, len(_score)):
                    _score[idx] = ran.randrange(5, 21)
        else:
            if 'temp' in self.improved_features:
                if (temp >= 18) & (temp <= 20):
                    _score[0] = 20
                else:
                    err = 0
                    if temp <= 18:
                        err = 18 - temp
                    else:
                        err = temp - 20
                    _score[0] = 20 - round(err / 2)
            else:
                _score[0] = ran.randrange(16, 21)
                    

            if 'hum' in self.improved_features:
                if (hum >= 40) & (hum <= 60):
                    _score[1] = 20
                else:
                    err = 0
                    if hum <= 40:
                        err = 40 - hum
                    else:
                        err = hum - 60
                    _score[1] = 20 - round(err / 5)
            else:
                _score[1] = ran.randrange(16, 21)


            if 'lux' in self.improved_features:
                if (lux >= 700) & (lux <= 1500):
                    _score[2] = 20
                else:
                    err = 0
                    if temp < 700:
                        err = 700 - lux
                    else:
                        err = lux - 1500
                    _score[2] = 20 - round(err / 100)
            else:
                _score[2] = ran.randrange(16, 21)

        return _score.sum()
    
def generate_features():
    features = ['temp', 'hum', 'lux']
    imp_features = list()
    for f in features:
        is_in = ran.randrange(0,2)
        if is_in == 0:
            imp_features.append(f)
    
    if len(imp_features) == 0:
        is_in = ran.randrange(0, 3)
        imp_features.append(features[is_in])
        
    return imp_features            

In [192]:
# improved_features - temp, hum, lux
case1 = User(1,['temp', 'hum'])
case1_score = case1.score(18, 30, 600)

now = dt.datetime.now()

now_after = now + dt.timedelta(days=1)
str_after = dt.datetime.strftime(now_after, '%Y-%m-%d')

case1.save_survey(18,30,600,case1_score, str_after)
case1.survey

,temp,hum,lux,datetime,score
0,18,30,600,2022-02-19,55


In [193]:
# 성실 참여자 20명 생성
users = list()
for user_id in range(0, 20):
    user = User(user_id, generate_features())
    users.append(user)
    
# 불성실 참여자 10명 생성
for user_id in range(20, 30):
    user = User(user_id, [])
    users.append(user)
    
now = dt.datetime.now()

for days in range(1, 366):
    now_after = now + dt.timedelta(days=days)
    str_after = dt.datetime.strftime(now_after, '%Y-%m-%d')
    
    # 온도 10~32
    ran_temp = ran.randrange(10, 33)
    
    # 습도 30 ~ 80
    ran_hum = ran.randrange(30, 81)
    
    # 조도 200 ~ 1900 (100 단위)
    ran_lux = ran.randrange(200, 1901, 100)
    
    for user in users:
        _s = user.score(ran_temp,
                       ran_hum,
                       ran_lux)
        user.save_survey(
            ran_temp,
            ran_hum,
            ran_lux,
            _s,
            str_after
        )

In [194]:
users[0].survey

,temp,hum,lux,datetime,score
0,18,35,300,2022-02-19,56
1,29,72,1000,2022-02-20,52
2,13,32,900,2022-02-21,50
3,15,64,700,2022-02-22,57
4,14,48,1600,2022-02-23,54
...,...,...,...,...,...
360,26,35,1600,2023-02-14,50
361,11,45,400,2023-02-15,53
362,24,53,1200,2023-02-16,54
363,16,62,800,2023-02-17,54


['temp']